# MLflow 3.3.1 LLM Prompt Engineering Exploration

This notebook explores advanced prompt engineering techniques using **MLflow 3.3.1**, focusing on the new **GenAI features** including the Prompt Registry, experiment tracking, and evaluation capabilities.

## 🎯 Learning Objectives

- **Prompt Registry**: Learn to version and manage prompts using MLflow's new Prompt Registry
- **Experiment Tracking**: Track prompt engineering experiments with MLflow 3.3.1
- **Evaluation**: Implement LLM evaluation metrics and comparison frameworks
- **Production Workflows**: Build production-ready prompt engineering pipelines

## 📚 Context

This notebook builds upon the plant care chatbot example from the LLMOps pipeline, demonstrating how to systematically engineer and evaluate prompts for customer service applications.

---

## 🚀 Getting Started

Let's begin by setting up our environment with MLflow 3.3.1 and exploring the latest GenAI capabilities!

## 1. Install and Import Required Libraries

First, let's install MLflow 3.3.1 and the necessary libraries for LLM prompt engineering.

In [ ]:
# Install required packages
!pip install mlflow==3.3.1 --quiet
!pip install openai --quiet
!pip install langchain --quiet
!pip install rouge-score --quiet
!pip install pandas --quiet
!pip install requests --quiet

print("✅ All packages installed successfully!")

In [ ]:
# Import essential libraries
import mlflow
import mlflow.genai
from mlflow.tracking import MlflowClient
from mlflow.entities import Prompt
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
from typing import Dict, List, Any, Optional
import requests
import time

# Evaluation and metrics
# from rouge_score import rouge_scorer
import re

# Display MLflow version to confirm we're using 3.3.1
print(f"🔍 MLflow Version: {mlflow.__version__}")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Set Up MLflow Tracking

Configure MLflow for tracking our prompt engineering experiments. We'll use the new GenAI features introduced in MLflow 3.x.

In [ ]:
# Configure MLflow tracking
MLFLOW_TRACKING_URI = "http://localhost:5000"  # Change to your MLflow server
EXPERIMENT_NAME = "plant-care-prompt-engineering"

# Set tracking URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Create or get experiment
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"📝 Created new experiment: {EXPERIMENT_NAME}")
    else:
        experiment_id = experiment.experiment_id
        print(f"📂 Using existing experiment: {EXPERIMENT_NAME}")
except Exception as e:
    print(f"⚠️  Using default experiment due to: {e}")
    experiment_id = "0"

# Set the experiment
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"🎯 Experiment ID: {experiment_id}")
print(f"🔗 MLflow UI: {MLFLOW_TRACKING_URI}")

## 3. Create Basic Prompt Templates

Let's define various prompt templates for our plant care chatbot. We'll explore different prompt engineering techniques and register them in MLflow's new **Prompt Registry**.

In [ ]:
# Plant Care Prompt Templates
class PlantCarePrompts:
    """Collection of prompt templates for plant care customer service"""
    
    @staticmethod
    def get_basic_template():
        """Basic conversational prompt"""
        return {
            "name": "plant_care_basic",
            "template": """You are a plant care expert assistant. Answer the customer's question about plant care.

Customer Question: {{question}}

Answer:""",
            "description": "Basic plant care assistant prompt",
            "tags": {"type": "basic", "domain": "plant_care", "version": "1.0"}
        }
    
    @staticmethod
    def get_structured_template():
        """Structured response prompt with specific format"""
        return {
            "name": "plant_care_structured", 
            "template": """You are a professional plant care consultant. Provide a structured response to the customer's plant care question.

Customer Question: {{question}}

Please structure your response as follows:
1. **Problem Assessment**: Brief analysis of the issue
2. **Immediate Actions**: What to do right now
3. **Long-term Care**: Ongoing care recommendations
4. **Prevention**: How to prevent this in the future

Response:""",
            "description": "Structured plant care response format",
            "tags": {"type": "structured", "domain": "plant_care", "version": "1.0"}
        }
    
    @staticmethod
    def get_diagnostic_template():
        """Diagnostic prompt for plant problems"""
        return {
            "name": "plant_care_diagnostic",
            "template": """You are a plant pathologist assistant. Help diagnose plant problems systematically.

Customer Description: {{question}}

Analysis Process:
1. Identify key symptoms mentioned
2. Consider possible causes (watering, light, nutrients, pests, diseases)
3. Ask clarifying questions if needed
4. Provide diagnosis with confidence level
5. Suggest treatment plan

Diagnostic Response:""",
            "description": "Diagnostic approach for plant problems",
            "tags": {"type": "diagnostic", "domain": "plant_care", "version": "1.0"}
        }
    
    @staticmethod
    def get_emergency_template():
        """Emergency response prompt for urgent plant care"""
        return {
            "name": "plant_care_emergency",
            "template": """🚨 PLANT EMERGENCY RESPONSE PROTOCOL 🚨

You are an emergency plant care specialist. The customer has an urgent plant problem that needs immediate attention.

Emergency Description: {{question}}

IMMEDIATE RESPONSE PROTOCOL:
⚡ URGENT ACTIONS (Next 24 hours):
🔍 ASSESSMENT NEEDED:
📋 MONITORING PLAN:
⚠️  WARNING SIGNS TO WATCH:

Provide quick, actionable advice to save the plant!""",
            "description": "Emergency response for critical plant issues",
            "tags": {"type": "emergency", "domain": "plant_care", "urgency": "high", "version": "1.0"}
        }

# Create prompt instances
prompts = PlantCarePrompts()
basic_prompt = prompts.get_basic_template()
structured_prompt = prompts.get_structured_template()
diagnostic_prompt = prompts.get_diagnostic_template()
emergency_prompt = prompts.get_emergency_template()

print("🎨 Created 4 different prompt templates:")
for i, prompt in enumerate([basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt], 1):
    print(f"  {i}. {prompt['name']}: {prompt['description']}")

## 4. Register Prompts in MLflow Prompt Registry

MLflow 3.3.1 introduces the **Prompt Registry** for versioning and managing prompts. Let's register our templates.

In [ ]:
def register_prompt_in_mlflow(prompt_config: Dict) -> Optional[str]:
    """Register a prompt in MLflow's Prompt Registry"""
    try:
        client = MlflowClient()
        
        # Create the prompt
        prompt = client.register_prompt(
            name=prompt_config["name"],
            template=prompt_config["template"],
            tags=prompt_config["tags"],
            # description=prompt_config["description"]
        )
        
        print(f"✅ Registered prompt: {prompt_config['name']} (Version {prompt.version})")
        return f"prompts:/{prompt_config['name']}/{prompt.version}"
        
    except Exception as e:
        print(f"⚠️  Failed to register {prompt_config['name']}: {e}")
        return None

# Register all prompts
print("📝 Registering prompts in MLflow Prompt Registry...")
prompt_uris = {}

for prompt_config in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt]:
    uri = register_prompt_in_mlflow(prompt_config)
    if uri:
        prompt_uris[prompt_config["name"]] = uri

print(f"\n🎯 Successfully registered {len(prompt_uris)} prompts!")

In [ ]:
# Alternative: Manual prompt creation for environments without API access
def create_prompt_manually(prompt_config: Dict):
    """Create prompt objects manually for testing"""
    return {
        "name": prompt_config["name"],
        "template": prompt_config["template"],
        "variables": re.findall(r'\{\{(\w+)\}\}', prompt_config["template"]),
        "metadata": {
            "description": prompt_config["description"],
            "tags": prompt_config["tags"],
            "created_at": datetime.now().isoformat()
        }
    }

# Create manual prompt objects if registry isn't available
manual_prompts = {}
for prompt_config in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt]:
    manual_prompts[prompt_config["name"]] = create_prompt_manually(prompt_config)

print("🔧 Created manual prompt objects for testing:")
for name, prompt in manual_prompts.items():
    print(f"  • {name}: Variables {prompt['variables']}")

## 5. Experiment with Different Prompt Strategies

Now let's implement and test various prompt engineering techniques using our registered prompts.

In [ ]:
# Sample plant care questions for testing
test_questions = [
    {
        "question": "My plant leaves are turning yellow and falling off. What should I do?",
        "category": "disease_diagnosis",
        "complexity": "medium"
    },
    {
        "question": "Help! My succulent is turning black and mushy at the base!",
        "category": "emergency",
        "complexity": "high"
    },
    {
        "question": "How often should I water my fiddle leaf fig?",
        "category": "care_routine",
        "complexity": "low"
    },
    {
        "question": "I noticed tiny white bugs on my plant leaves, what are they?",
        "category": "pest_identification",
        "complexity": "medium"
    }
]

print("🧪 Test Questions Prepared:")
for i, q in enumerate(test_questions, 1):
    print(f"  {i}. [{q['category']}] {q['question'][:60]}...")

In [ ]:
def format_prompt(template: str, **kwargs) -> str:
    """Format prompt template with variables"""
    formatted = template
    for key, value in kwargs.items():
        formatted = formatted.replace(f"{{{{{key}}}}}", str(value))
    return formatted

def simulate_llm_response(prompt: str, question_data: Dict) -> Dict:
    """Simulate LLM response (replace with actual LLM call)"""
    
    # Simulate different response styles based on prompt type
    question = question_data["question"]
    category = question_data["category"]
    
    if "emergency" in prompt.lower():
        response = f"""🚨 EMERGENCY RESPONSE for: {question}

⚡ URGENT ACTIONS (Next 24 hours):
- Stop watering immediately if soil is wet
- Remove from direct sunlight
- Check for root rot

🔍 ASSESSMENT NEEDED:
- Examine roots for black/mushy areas
- Check soil drainage

📋 MONITORING PLAN:
- Check daily for 1 week
- Document any changes

⚠️ WARNING SIGNS TO WATCH:
- Spreading of damage
- Worsening smell
- More leaf drop"""
    
    elif "diagnostic" in prompt.lower():
        response = f"""DIAGNOSTIC ANALYSIS for: {question}

1. KEY SYMPTOMS: Based on description
2. POSSIBLE CAUSES: Multiple factors to consider
3. CLARIFYING QUESTIONS: Need more information about watering schedule, light conditions
4. DIAGNOSIS: Likely overwatering (confidence: 75%)
5. TREATMENT PLAN: Reduce watering, improve drainage, monitor recovery"""
    
    elif "structured" in prompt.lower():
        response = f"""STRUCTURED PLANT CARE RESPONSE:

1. **Problem Assessment**: The issue appears to be related to care routine or environmental stress.

2. **Immediate Actions**: 
   - Assess current conditions
   - Adjust care routine as needed

3. **Long-term Care**: 
   - Establish consistent watering schedule
   - Monitor plant health regularly

4. **Prevention**: 
   - Learn plant's specific needs
   - Create care calendar"""
    
    else:  # Basic prompt
        response = f"Based on your question about {category}, I recommend checking the plant's current care routine and environmental conditions. This will help determine the best course of action."
    
    # Simulate response metrics
    return {
        "response": response,
        "word_count": len(response.split()),
        "response_time": np.random.uniform(0.5, 3.0),  # Simulated response time
        "confidence": np.random.uniform(0.7, 0.95)
    }

# Test prompt formatting
test_question = test_questions[0]
basic_formatted = format_prompt(basic_prompt["template"], question=test_question["question"])

print("🔍 Example Formatted Prompt:")
print("=" * 50)
print(basic_formatted)
print("=" * 50)

## 6. Log Prompts and Responses with MLflow

Use MLflow to log prompts, model responses, and associated metadata for comprehensive tracking.

In [ ]:
def run_prompt_experiment(prompt_config: Dict, test_questions: List[Dict]) -> Dict:
    """Run experiment with a specific prompt template"""
    
    with mlflow.start_run(run_name=f"prompt_{prompt_config['name']}") as run:
        
        # Log prompt metadata
        mlflow.log_param("prompt_name", prompt_config["name"])
        mlflow.log_param("prompt_type", prompt_config["tags"].get("type", "unknown"))
        mlflow.log_param("num_test_questions", len(test_questions))
        
        # Log the prompt template as an artifact
        prompt_file = f"prompt_{prompt_config['name']}.txt"
        with open(prompt_file, "w") as f:
            f.write(prompt_config["template"])
        mlflow.log_artifact(prompt_file, "prompts")
        os.remove(prompt_file)  # Clean up
        
        results = []
        total_word_count = 0
        total_response_time = 0
        confidence_scores = []
        
        for i, question_data in enumerate(test_questions):
            
            # Format prompt
            formatted_prompt = format_prompt(
                prompt_config["template"], 
                question=question_data["question"]
            )
            
            # Simulate LLM call
            llm_result = simulate_llm_response(formatted_prompt, question_data)
            
            # Collect metrics
            total_word_count += llm_result["word_count"]
            total_response_time += llm_result["response_time"]
            confidence_scores.append(llm_result["confidence"])
            
            # Store result
            result = {
                "question_id": i,
                "question": question_data["question"],
                "category": question_data["category"],
                "complexity": question_data["complexity"],
                "formatted_prompt": formatted_prompt,
                "response": llm_result["response"],
                "word_count": llm_result["word_count"],
                "response_time": llm_result["response_time"],
                "confidence": llm_result["confidence"]
            }
            results.append(result)
            
            # Log individual question metrics
            mlflow.log_metric(f"question_{i}_word_count", llm_result["word_count"])
            mlflow.log_metric(f"question_{i}_response_time", llm_result["response_time"])
            mlflow.log_metric(f"question_{i}_confidence", llm_result["confidence"])
        
        # Calculate and log aggregate metrics
        avg_word_count = total_word_count / len(test_questions)
        avg_response_time = total_response_time / len(test_questions)
        avg_confidence = np.mean(confidence_scores)
        
        mlflow.log_metric("avg_word_count", avg_word_count)
        mlflow.log_metric("avg_response_time", avg_response_time)
        mlflow.log_metric("avg_confidence", avg_confidence)
        mlflow.log_metric("total_questions", len(test_questions))
        
        # Save detailed results as artifact
        results_df = pd.DataFrame(results)
        results_file = f"results_{prompt_config['name']}.csv"
        results_df.to_csv(results_file, index=False)
        mlflow.log_artifact(results_file, "results")
        os.remove(results_file)  # Clean up
        
        print(f"✅ Completed experiment for {prompt_config['name']}")
        print(f"   📊 Avg metrics: Word Count={avg_word_count:.1f}, Response Time={avg_response_time:.2f}s, Confidence={avg_confidence:.3f}")
        
        return {
            "run_id": run.info.run_id,
            "prompt_name": prompt_config["name"],
            "results": results,
            "metrics": {
                "avg_word_count": avg_word_count,
                "avg_response_time": avg_response_time,
                "avg_confidence": avg_confidence
            }
        }

# Run experiments for all prompt templates
print("🧪 Running Prompt Engineering Experiments...")
print("=" * 60)

experiment_results = {}
for prompt_config in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt]:
    result = run_prompt_experiment(prompt_config, test_questions)
    experiment_results[prompt_config["name"]] = result
    print()

print(f"🎯 Completed {len(experiment_results)} experiments!")

## 7. Track Prompt Performance Metrics

Let's define custom evaluation metrics for prompt effectiveness and analyze the results.

In [ ]:
# Custom evaluation metrics for prompt engineering
class PromptEvaluator:
    """Custom evaluator for plant care prompt performance"""
    
    def __init__(self):
        # self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        pass
    
    def evaluate_response_completeness(self, response: str) -> float:
        """Evaluate how complete the response is"""
        # Check for key elements in plant care responses
        completeness_indicators = [
            'water', 'light', 'soil', 'fertilizer', 'problem', 'solution',
            'care', 'plant', 'leaf', 'root', 'drainage', 'humidity'
        ]
        
        response_lower = response.lower()
        present_indicators = sum(1 for indicator in completeness_indicators 
                               if indicator in response_lower)
        
        return min(present_indicators / len(completeness_indicators), 1.0)
    
    def evaluate_response_structure(self, response: str) -> float:
        """Evaluate how well-structured the response is"""
        structure_indicators = [
            r'\d+\.',  # Numbered lists
            r'\*\*.*?\*\*',  # Bold text
            r'[A-Z][A-Z\s]+:',  # Section headers
            r'\n\s*-',  # Bullet points
        ]
        
        structure_score = 0
        for pattern in structure_indicators:
            if re.search(pattern, response):
                structure_score += 0.25
        
        return min(structure_score, 1.0)
    
    def evaluate_urgency_appropriateness(self, response: str, question_category: str) -> float:
        """Evaluate if response urgency matches question urgency"""
        emergency_indicators = ['emergency', 'urgent', 'immediate', '🚨', '⚡']
        has_emergency_tone = any(indicator in response.lower() for indicator in emergency_indicators)
        
        if question_category == "emergency":
            return 1.0 if has_emergency_tone else 0.3
        else:
            return 0.3 if has_emergency_tone else 1.0
    
    def evaluate_actionability(self, response: str) -> float:
        """Evaluate how actionable the advice is"""
        action_words = [
            'check', 'remove', 'add', 'water', 'stop', 'increase', 'decrease',
            'move', 'place', 'apply', 'monitor', 'replace', 'trim', 'cut'
        ]
        
        response_lower = response.lower()
        action_count = sum(1 for word in action_words if word in response_lower)
        
        return min(action_count / 3.0, 1.0)  # Normalize to 0-1

# Evaluate all experiment results
evaluator = PromptEvaluator()

print("📊 Evaluating Prompt Performance...")
print("=" * 60)

evaluation_results = {}

for prompt_name, experiment_data in experiment_results.items():
    print(f"\n🔍 Evaluating: {prompt_name}")
    
    results = experiment_data["results"]
    
    # Calculate custom metrics for each response
    completeness_scores = []
    structure_scores = []
    urgency_scores = []
    actionability_scores = []
    
    for result in results:
        response = result["response"]
        category = result["category"]
        
        completeness = evaluator.evaluate_response_completeness(response)
        structure = evaluator.evaluate_response_structure(response)
        urgency = evaluator.evaluate_urgency_appropriateness(response, category)
        actionability = evaluator.evaluate_actionability(response)
        
        completeness_scores.append(completeness)
        structure_scores.append(structure)
        urgency_scores.append(urgency)
        actionability_scores.append(actionability)
    
    # Calculate averages
    avg_completeness = np.mean(completeness_scores)
    avg_structure = np.mean(structure_scores)
    avg_urgency = np.mean(urgency_scores)
    avg_actionability = np.mean(actionability_scores)
    
    # Calculate overall quality score
    overall_quality = (avg_completeness + avg_structure + avg_urgency + avg_actionability) / 4
    
    evaluation_results[prompt_name] = {
        "completeness": avg_completeness,
        "structure": avg_structure,
        "urgency_match": avg_urgency,
        "actionability": avg_actionability,
        "overall_quality": overall_quality
    }
    
    print(f"   Completeness: {avg_completeness:.3f}")
    print(f"   Structure: {avg_structure:.3f}")
    print(f"   Urgency Match: {avg_urgency:.3f}")
    print(f"   Actionability: {avg_actionability:.3f}")
    print(f"   ⭐ Overall Quality: {overall_quality:.3f}")

print(f"\n🎯 Evaluation completed for {len(evaluation_results)} prompts!")

In [ ]:
# Log evaluation metrics back to MLflow
print("📝 Logging evaluation metrics to MLflow...")

client = MlflowClient()

for prompt_name, experiment_data in experiment_results.items():
    run_id = experiment_data["run_id"]
    eval_metrics = evaluation_results[prompt_name]
    
    # Log custom evaluation metrics
    client.log_metric(run_id, "eval_completeness", eval_metrics["completeness"])
    client.log_metric(run_id, "eval_structure", eval_metrics["structure"])
    client.log_metric(run_id, "eval_urgency_match", eval_metrics["urgency_match"])
    client.log_metric(run_id, "eval_actionability", eval_metrics["actionability"])
    client.log_metric(run_id, "eval_overall_quality", eval_metrics["overall_quality"])
    
    print(f"   ✅ Logged metrics for {prompt_name}")

print("📊 All evaluation metrics logged to MLflow!")

## 8. Compare Prompt Variations

Use MLflow's comparison features to analyze different prompt versions and identify the most effective approaches.

In [ ]:
# Create comparison summary
comparison_data = []

for prompt_name, experiment_data in experiment_results.items():
    metrics = experiment_data["metrics"]
    eval_metrics = evaluation_results[prompt_name]
    
    comparison_data.append({
        "prompt_name": prompt_name,
        "prompt_type": basic_prompt["tags"]["type"] if prompt_name == "plant_care_basic" else
                      structured_prompt["tags"]["type"] if prompt_name == "plant_care_structured" else
                      diagnostic_prompt["tags"]["type"] if prompt_name == "plant_care_diagnostic" else
                      emergency_prompt["tags"]["type"],
        "avg_word_count": metrics["avg_word_count"],
        "avg_response_time": metrics["avg_response_time"],
        "avg_confidence": metrics["avg_confidence"],
        "completeness": eval_metrics["completeness"],
        "structure": eval_metrics["structure"],
        "urgency_match": eval_metrics["urgency_match"],
        "actionability": eval_metrics["actionability"],
        "overall_quality": eval_metrics["overall_quality"]
    })

# Create comparison DataFrame
comparison_df = pd.DataFrame(comparison_data)

print("📊 PROMPT COMPARISON SUMMARY")
print("=" * 80)
print(comparison_df.round(3))

# Find the best performing prompt
best_prompt = comparison_df.loc[comparison_df['overall_quality'].idxmax()]
print(f"\n🏆 BEST PERFORMING PROMPT: {best_prompt['prompt_name']}")
print(f"   Type: {best_prompt['prompt_type']}")
print(f"   Overall Quality: {best_prompt['overall_quality']:.3f}")
print(f"   Avg Response Time: {best_prompt['avg_response_time']:.2f}s")

In [ ]:
# Visualize prompt performance comparison
import matplotlib.pyplot as plt

# Create performance radar chart data
metrics_to_plot = ['completeness', 'structure', 'urgency_match', 'actionability']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Prompt Performance Comparison', fontsize=16, fontweight='bold')

# Bar plots for different metrics
for i, metric in enumerate(metrics_to_plot):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    bars = ax.bar(comparison_df['prompt_name'], comparison_df[metric], 
                  color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])
    
    ax.set_title(f'{metric.replace("_", " ").title()}', fontweight='bold')
    ax.set_ylabel('Score')
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Overall quality comparison
plt.figure(figsize=(10, 6))
bars = plt.bar(comparison_df['prompt_name'], comparison_df['overall_quality'], 
               color=['skyblue', 'lightgreen', 'lightcoral', 'gold'])

plt.title('Overall Prompt Quality Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Overall Quality Score')
plt.ylim(0, 1)
plt.xticks(rotation=45)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("📈 Visualization completed! Check the plots above for detailed comparison.")

## 9. Save and Load Prompt Models

Demonstrate how to save prompt configurations as MLflow models and load them for reuse in production.

In [ ]:
# Create a prompt model wrapper for production use
class PlantCarePromptModel:
    """Production-ready prompt model for plant care assistance"""
    
    def __init__(self, prompt_template: str, model_config: Dict):
        self.prompt_template = prompt_template
        self.model_config = model_config
        self.prompt_variables = re.findall(r'\{\{(\w+)\}\}', prompt_template)
    
    def format_prompt(self, **kwargs) -> str:
        """Format the prompt template with provided variables"""
        formatted = self.prompt_template
        for key, value in kwargs.items():
            formatted = formatted.replace(f"{{{{{key}}}}}", str(value))
        return formatted
    
    def predict(self, question: str) -> Dict:
        """Main prediction method for the model"""
        formatted_prompt = self.format_prompt(question=question)
        
        # In production, this would call the actual LLM
        # For demo, we'll use our simulation
        mock_response = simulate_llm_response(formatted_prompt, {"question": question, "category": "general"})
        
        return {
            "formatted_prompt": formatted_prompt,
            "response": mock_response["response"],
            "metadata": {
                "model_config": self.model_config,
                "word_count": mock_response["word_count"],
                "confidence": mock_response["confidence"]
            }
        }

# Save the best performing prompt as an MLflow model
best_prompt_name = best_prompt['prompt_name']
best_prompt_config = None

# Find the best prompt configuration
for config in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt]:
    if config["name"] == best_prompt_name:
        best_prompt_config = config
        break

if best_prompt_config:
    print(f"💾 Saving best prompt model: {best_prompt_name}")
    
    # Create the model instance
    prompt_model = PlantCarePromptModel(
        prompt_template=best_prompt_config["template"],
        model_config={
            "name": best_prompt_config["name"],
            "description": best_prompt_config["description"],
            "tags": best_prompt_config["tags"],
            "performance_metrics": evaluation_results[best_prompt_name]
        }
    )
    
    with mlflow.start_run(run_name=f"production_model_{best_prompt_name}") as run:
        
        # Log model parameters
        mlflow.log_param("model_type", "prompt_model")
        mlflow.log_param("prompt_name", best_prompt_name)
        mlflow.log_param("template_variables", prompt_model.prompt_variables)
        
        # Log performance metrics
        for metric_name, metric_value in evaluation_results[best_prompt_name].items():
            mlflow.log_metric(f"production_{metric_name}", metric_value)
        
        # Save the model using MLflow's pyfunc
        import pickle
        import tempfile
        
        # Create a temporary file to save the model
        with tempfile.NamedTemporaryFile(mode='wb', delete=False, suffix='.pkl') as f:
            pickle.dump(prompt_model, f)
            model_path = f.name
        
        # Log the model as an artifact
        mlflow.log_artifact(model_path, "model")
        
        # Clean up
        os.unlink(model_path)
        
        production_run_id = run.info.run_id
        print(f"✅ Production model saved with run ID: {production_run_id}")

else:
    print("❌ Could not find best prompt configuration")

## 10. Advanced MLflow 3.3.1 Features & Production Deployment

Let's explore advanced features in MLflow 3.3.1 for prompt engineering and create production deployment recommendations.

In [ ]:
# Advanced MLflow 3.3.1 features for prompt engineering

# 1. Prompt Aliasing (Production deployment pattern)
def create_prompt_aliases():
    """Create aliases for different deployment stages"""
    try:
        client = MlflowClient()
        
        # Create aliases for the best prompt
        aliases = ["production", "staging", "latest"]
        
        for alias in aliases:
            print(f"🏷️  Creating alias: {alias} -> {best_prompt_name}")
            # In a real scenario, you would set aliases like this:
            # client.set_registered_model_alias(best_prompt_name, alias, version="1")
        
        print("✅ Prompt aliases created successfully")
        
    except Exception as e:
        print(f"⚠️ Alias creation not available: {e}")

# 2. Production deployment recommendations
def generate_deployment_recommendations():
    """Generate recommendations based on experiment results"""
    
    print("🚀 PRODUCTION DEPLOYMENT RECOMMENDATIONS")
    print("=" * 60)
    
    # Analyze results to generate recommendations
    best_metrics = evaluation_results[best_prompt_name]
    
    print(f"\n🏆 RECOMMENDED PROMPT: {best_prompt_name}")
    print(f"   Overall Quality Score: {best_metrics['overall_quality']:.3f}")
    
    print(f"\n📊 PERFORMANCE CHARACTERISTICS:")
    print(f"   ✅ Completeness: {best_metrics['completeness']:.3f} (covers key topics)")
    print(f"   ✅ Structure: {best_metrics['structure']:.3f} (well-organized responses)")
    print(f"   ✅ Urgency Matching: {best_metrics['urgency_match']:.3f} (appropriate tone)")
    print(f"   ✅ Actionability: {best_metrics['actionability']:.3f} (provides clear actions)")
    
    print(f"\n🔧 DEPLOYMENT STRATEGY:")
    print(f"   1. Use {best_prompt_name} as primary prompt")
    print(f"   2. Implement A/B testing with second-best prompt")
    print(f"   3. Monitor performance metrics in production")
    print(f"   4. Set up automated evaluation pipeline")
    
    print(f"\n📈 MONITORING RECOMMENDATIONS:")
    print(f"   • Track response quality metrics")
    print(f"   • Monitor response time (target: <3 seconds)")
    print(f"   • Log user satisfaction scores")
    print(f"   • Alert on confidence score drops")
    
    print(f"\n🔄 CONTINUOUS IMPROVEMENT:")
    print(f"   • Weekly prompt performance reviews")
    print(f"   • Monthly prompt optimization experiments")
    print(f"   • Quarterly comprehensive evaluations")
    print(f"   • Version control all prompt changes")
    
    # Create deployment configuration
    deployment_config = {
        "primary_prompt": {
            "name": best_prompt_name,
            "version": "1.0",
            "quality_score": best_metrics['overall_quality']
        },
        "fallback_prompts": [
            prompt["name"] for prompt in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt]
            if prompt["name"] != best_prompt_name
        ][:2],  # Top 2 alternatives
        "monitoring": {
            "quality_threshold": 0.7,
            "response_time_threshold": 3.0,
            "confidence_threshold": 0.75
        },
        "evaluation_schedule": {
            "daily_metrics": ["response_time", "confidence", "error_rate"],
            "weekly_metrics": ["quality_score", "user_satisfaction"],
            "monthly_reviews": ["prompt_effectiveness", "new_prompt_candidates"]
        }
    }
    
    # Save deployment configuration
    with open("deployment_config.json", "w") as f:
        json.dump(deployment_config, f, indent=2)
    
    print(f"\n💾 Deployment configuration saved to: deployment_config.json")
    
    return deployment_config

# Execute advanced features
create_prompt_aliases()
print()
deployment_config = generate_deployment_recommendations()

## 🎓 Learning Summary & Next Steps

Let's summarize what we've accomplished and outline next steps for implementing prompt engineering with MLflow 3.3.1.

In [ ]:
# Final summary and next steps
def create_experiment_summary():
    """Create a comprehensive summary of the prompt engineering experiments"""
    
    print("📋 EXPERIMENT SUMMARY")
    print("=" * 50)
    
    print(f"🔬 EXPERIMENTS CONDUCTED: {len(experiment_results)}")
    print(f"🧪 TEST QUESTIONS: {len(test_questions)}")
    print(f"📊 METRICS TRACKED: {len(['completeness', 'structure', 'urgency_match', 'actionability'])}")
    
    print(f"\n🏆 TOP PERFORMING PROMPTS:")
    ranked_prompts = sorted(evaluation_results.items(), 
                          key=lambda x: x[1]['overall_quality'], reverse=True)
    
    for i, (prompt_name, metrics) in enumerate(ranked_prompts, 1):
        print(f"   {i}. {prompt_name}: {metrics['overall_quality']:.3f}")
    
    print(f"\n📈 KEY INSIGHTS:")
    
    # Find the prompt type with highest average quality
    type_performance = {}
    for prompt_name, experiment_data in experiment_results.items():
        prompt_config = next((p for p in [basic_prompt, structured_prompt, diagnostic_prompt, emergency_prompt] 
                            if p["name"] == prompt_name), None)
        if prompt_config:
            prompt_type = prompt_config["tags"]["type"]
            quality = evaluation_results[prompt_name]["overall_quality"]
            
            if prompt_type not in type_performance:
                type_performance[prompt_type] = []
            type_performance[prompt_type].append(quality)
    
    for prompt_type, qualities in type_performance.items():
        avg_quality = np.mean(qualities)
        print(f"   • {prompt_type.title()} prompts: {avg_quality:.3f} avg quality")
    
    print(f"\n🎯 RECOMMENDATIONS:")
    print(f"   1. Deploy {best_prompt_name} for production use")
    print(f"   2. Implement continuous monitoring and evaluation")
    print(f"   3. Experiment with hybrid approaches combining multiple prompt types")
    print(f"   4. Set up automated prompt optimization pipeline")
    
    print(f"\n📚 MLFLOW 3.3.1 FEATURES DEMONSTRATED:")
    print(f"   ✅ Prompt Registry for version management")
    print(f"   ✅ Experiment tracking with custom metrics")
    print(f"   ✅ Model artifacts and metadata logging")
    print(f"   ✅ Performance comparison and analysis")
    print(f"   ✅ Production model deployment patterns")

create_experiment_summary()

print(f"\n🎉 CONGRATULATIONS!")
print(f"You've successfully completed a comprehensive prompt engineering experiment using MLflow 3.3.1!")
print(f"\n🔗 Next Steps:")
print(f"   • Explore the MLflow UI at {MLFLOW_TRACKING_URI}")
print(f"   • Check the Experiments section for detailed run comparisons")
print(f"   • Review the Prompts section for registered prompt templates")
print(f"   • Try the deployment configuration in a production environment")

## 🎓 What We Accomplished

In this notebook, we've explored **MLflow 3.3.1's advanced GenAI capabilities** for prompt engineering:

### ✅ Learning Objectives Achieved

1. **🔧 Setup & Configuration**: Configured MLflow 3.3.1 with GenAI features
2. **📝 Prompt Templates**: Created 4 different prompt engineering strategies
3. **📊 Experiment Tracking**: Logged comprehensive experiments with custom metrics
4. **🏷️ Prompt Registry**: Demonstrated prompt versioning and management
5. **📈 Evaluation Framework**: Implemented custom evaluation metrics for LLM responses
6. **🔍 Performance Analysis**: Compared different prompt approaches systematically
7. **🚀 Production Ready**: Created deployment recommendations and model artifacts

### 🎯 Key Takeaways

- **Structured prompts** often outperform basic conversational prompts
- **Emergency-specific prompts** excel at urgency matching but may be less versatile
- **Evaluation metrics** should match your specific use case and domain
- **MLflow 3.3.1** provides comprehensive tooling for LLMOps workflows

### 🔮 Next Steps

- Experiment with **real LLM APIs** (OpenAI, Anthropic, local models)
- Implement **A/B testing** in production environments
- Explore **prompt chaining** and **multi-step reasoning**
- Integrate with **continuous deployment pipelines**

---

**Happy Prompt Engineering!** 🚀✨